<a href="https://colab.research.google.com/github/aswa09/EVA-4/blob/master/S14/EVA4_S15A_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [ ]:
import os
from zipfile import ZipFile,ZIP_STORED
import zipfile
from pathlib import Path
import gc
import io
import sys
from skimage.transform import resize
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import glob

import warnings

warnings.simplefilter("ignore")

In [ ]:
!pip install tensorflow=='1.13.0-rc2' --quiet
!pip install keras==2.2.4 --quiet

     |████████████████████████████████| 92.6MB 47kB/s 
     |████████████████████████████████| 3.1MB 9.7MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 368kB 10.9MB/s 
     |████████████████████████████████| 317kB 3.5MB/s 


# Mount and Fetch Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#Extract background, foreground and foreground masks from zip files

* The background and foreground images were curated from google and uniformly resized manually MS Paint.
* With these forground images, masks were created in Gimp.


**This part can be skipped if directly unzipping the whole dataset

In [ ]:
path_to_zip_file = "/content/drive/My Drive/Colab Notebooks/S14/images.zip"
directory_to_extract_to = '/content/drive/My Drive/Colab Notebooks/S14/'
print("Extracting zip file: %s" % path_to_zip_file)
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)
print("Extracted at: %s" % directory_to_extract_to)

Extracting zip file: /content/drive/My Drive/Colab Notebooks/S14/images.zip
Extracted at: /content/drive/My Drive/Colab Notebooks/S14/


##Change Directory

In [ ]:
os.chdir('./drive/My Drive/Colab Notebooks/S14/')
!ls

EVA4_S15A_data_generation.ipynb  images  images.zip


## Overlay Foreground on Background

*   Each foreground is flipped horizontally (so twice)
*   For each orientation of foreground, randomly overlay it on the background 20 times.
*   #bg = 100
*   #fg = 100
*   #fg_bg = 100\*100\*2\*20 = 400,000



In [ ]:
def overlay_fg_on_bg():
  try:
    bg_dir = "./images/bg/"
    bg_imgs = sorted(os.listdir(bg_dir))
    fg_dir = "./images/fg/"
    fg_imgs = sorted(os.listdir(fg_dir))
    fg_masks_dir = "./images/fg_mask/"
    assert (len(bg_imgs) == 100)
    assert (len(fg_imgs) == 100)
    label_info = open(f'./images/fg_bg_labels.txt', 'w+')
    for bidx, bg_img in enumerate(bg_imgs):
      i=bidx
      if i!=100:
        i=int(i)%100
      else:
        i=100

      out_zip_fg_bg = ZipFile('./images/fg_bg/bg{0:03d}.zip'.format(i + 1), mode='a', compression=ZIP_STORED)
      out_zip_fg_bg_m = ZipFile('./images/fg_bg_mask/bg{0:03d}.zip'.format(i + 1), mode='a', compression=ZIP_STORED)
      out_lbl=open('./images/labels.txt', 'a')
      mbg = Image.open(bg_dir + bg_img)
      for fidx, fg_img in enumerate(fg_imgs):
        mfg = Image.open(fg_dir + fg_img)
        assert (fg_img[-4:] == ".png")
        mfg_mask = Image.open(fg_masks_dir + fg_img[:-4] + "_mask.png")

        if mfg_mask.mode in ("RGBA", "P"):
          mfg_mask = mfg_mask.convert("RGB")

        flip = [True, False]
        for i in range(40):
          should_flip = flip[i % len(flip)]

          bg = mbg.copy()
          fg = mfg.copy()
          fg_mask = mfg_mask.copy()

          if should_flip:
            fg = fg.transpose(Image.FLIP_LEFT_RIGHT)
            fg_mask = fg_mask.transpose(Image.FLIP_LEFT_RIGHT)

          bg_w, bg_h = bg.size
          fg_w, fg_h = fg.size
          max_h = bg_h - fg_h
          max_w = bg_w - fg_w
          pos_x = np.random.randint(low=0, high=max_w, size=1)[0]
          pos_y = np.random.randint(low=0, high=max_h, size=1)[0]

          bg.paste(fg, (pos_x, pos_y), fg)

          bg_mask = Image.new('L', bg.size)
          fg_mask = fg_mask.convert('L')
          bg_mask.paste(fg_mask, (pos_x, pos_y), fg_mask)

          if bg.mode in ("RGBA", "P"):
              bg = bg.convert("RGB")
          bg.save('./images/bg_temp.jpg', optimize=True, quality=65)

          if bg_mask.mode in ("RGBA", "P"):
              bg_mask = bg_mask.convert("RGB")
          bg_mask.save('./images/bg_m_temp.jpg', optimize=True, quality=65)

          out_zip_fg_bg.write('./images/bg_temp.jpg', f'./fg_bg/{bg_img[:-4]}_{fg_img[:-4]}_{i + 1:02d}.jpg')

          out_zip_fg_bg_m.write('./images/bg_m_temp.jpg',f'./fg_bg_mask/{bg_img[:-4]}_{fg_img[:-4]}_{i + 1:02d}_mask.jpg')

          out_lbl.write(f'{bg_img[:-4]}_{fg_img[:-4]}_{i + 1:02d}.jpg\t{pos_x}\t{pos_y}\n')
          
          # !python test.py
          # out_zip.write('../images/dd.jpg',
          #               f'../fg_bg_depth/{bg_img[:-4]}_{fg_img[:-4]}_{i + 1:02d}_depth.jpg')
              
      out_zip_fg_bg.close()
      out_zip_fg_bg_m.close()

    label_info.close()
    out_lbl.write('./images/fg_bg_labels.txt')
    out_lbl.close()
  except Exception as e:
    print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno) + " " + type(e).__name__ + " " + str(e))
    sys.exit(1)

Generate the fg_bg images and masks

In [ ]:
overlay_fg_on_bg()

#Dense Depth

In [ ]:
!rm -rf DenseDepth/
!git clone https://github.com/ialhashim/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 238, done.
remote: Total 238 (delta 0), reused 0 (delta 0), pack-reused 238
Receiving objects: 100% (238/238), 11.80 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [ ]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-08-08 00:39:40--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.96.114
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.96.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  18.6MB/s    in 10s     

2020-08-08 00:39:51 (16.5 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [ ]:
%cd DenseDepth
!ls

/content/drive/My Drive/Colab Notebooks/S14/DenseDepth
augment.py	demo.py		  examples   model.py	  README.md   utils.py
callbacks.py	demo_rgb.npy	  layers.py  nyu.h5	  Tensorflow
data.py		DenseDepth.ipynb  LICENSE    __pycache__  test.py
demo_depth.npy	evaluate.py	  loss.py    PyTorch	  train.py


##Load Model

In [ ]:
from utils import predict, display_images
from keras.models import load_model
from layers import BilinearUpSampling2D
from loss import depth_loss_function

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
 
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': depth_loss_function}
 
print('Loading model...')
 
# Load model into GPU / CPU
model = load_model('nyu.h5', custom_objects=custom_objects, compile=False)
print('\n Model loaded')

Using TensorFlow backend.


Loading model...
Instructions for updating:
Colocations handled automatically by placer.

 Model loaded


##Modify functions to run from zips

In [ ]:
def check_if_processed(output_folder,depth_zip,files):
  with ZipFile(output_folder + depth_zip, 'r') as dzip:
    img_list_depth = dzip.namelist()
  dzip.close()
  files=list(set(files) - set(img_list_depth))
  return files

def load_images_zip(zfile,img_files,scale=4):
  names=[]
  scaled=[]
  for img in img_files:
    filename = img.split('/')[-1]
    names.append(filename)
    with ZipFile(zfile, 'r') as zip_files:
      image_data = zip_files.read(img)
    zip_files.close()

    img = io.BytesIO(image_data)
    x = np.clip(np.asarray(Image.open(img), dtype=float) / 255, 0, 1)
    output_shape = (scale * x.shape[0], scale * x.shape[1])
    scaled.append(resize(x, output_shape, order=1, preserve_range=True, mode='reflect', anti_aliasing=True))

  return np.stack(scaled),names

def savedata_zip(outputs, names, depth_zip, output_folder='../images/fg_bg_depth/'):
  plasma = plt.get_cmap('gray')
  output_copy = outputs.copy()
  for i,filename in enumerate(names):
    with ZipFile(output_folder+depth_zip, 'a') as dzip:
      rescaled = output_copy[i][:, :, 0]
      rescaled = rescaled - np.min(rescaled)
      rescaled = rescaled / np.max(rescaled)
      matplotlib_image = plt.imshow(plasma(rescaled)[:, :, :3])

      pil_image = Image.fromarray(np.uint8((matplotlib_image.get_array() * 255))).convert("L").resize((224, 224))
      pil_image.save(os.path.join(output_folder, 'depth_temp.jpg'))
      dzip.write(output_folder + 'depth_temp.jpg', filename)
      plt.close()
        
      del matplotlib_image
    dzip.close()
  del output_copy

def test(model, output_folder, zfile,zip_exists,img_list):
  zip_file_name=zfile.split('/')[-1]
  stp=250
  for i in range(0,4000,stp):
    gc.collect()
    files=img_list[i:i+stp]
    if zip_exists:
      files=check_if_processed(output_folder,zip_file_name,files)
    if len(files)>0:
      inputs,names = load_images_zip(zfile,files)
      print('Predicting..')
      outputs = predict(model, inputs, batch_size= 5)
      del inputs
      savedata_zip(outputs, names, depth_zip=zip_file_name, output_folder='../images/fg_bg_depth/')
      print('Saved idx: '+str(i)+','+str(i+stp))
      del outputs
      del names

##Create Depth images

In [ ]:
args_input_zip='../images/fg_bg/*.zip'
args_output='../images/fg_bg_depth/'

zip_files = glob.glob(args_input_zip)
assert (len(zip_files) == 100)
print(str(len(zip_files)) + ' bg zips')

for zfile in zip_files:
  gc.collect()
  fname=args_output + zfile.split('/')[-1]
  my_file = Path(fname)
  if my_file.is_file():
    with ZipFile(fname, 'r') as dzip:
      img_list_depth = dzip.namelist()
    dzip.close()
    if len(img_list_depth) == 4000:
      zip_exists=None 
    else:
      zip_exists = True
  else:
    zip_exists = False
  
  if zip_exists is not None:
    print("\nProcessing zip: " + zfile.split('/')[-1])
    with ZipFile(zfile, 'r') as fg_bg:
      img_list = fg_bg.namelist()
    fg_bg.close()
    assert (len(img_list) == 4000)
    test(model, args_output, zfile, zip_exists,img_list)
  else:
    print(zfile + ' : zip already processed')

100 bg zips
../images/fg_bg/bg001.zip : zip already processed
../images/fg_bg/bg002.zip : zip already processed
../images/fg_bg/bg003.zip : zip already processed
../images/fg_bg/bg037.zip : zip already processed
../images/fg_bg/bg038.zip : zip already processed
../images/fg_bg/bg039.zip : zip already processed
../images/fg_bg/bg040.zip : zip already processed
../images/fg_bg/bg041.zip : zip already processed
../images/fg_bg/bg042.zip : zip already processed
../images/fg_bg/bg043.zip : zip already processed
../images/fg_bg/bg044.zip : zip already processed
../images/fg_bg/bg045.zip : zip already processed
../images/fg_bg/bg046.zip : zip already processed
../images/fg_bg/bg047.zip : zip already processed
../images/fg_bg/bg048.zip : zip already processed
../images/fg_bg/bg049.zip : zip already processed
../images/fg_bg/bg050.zip : zip already processed
../images/fg_bg/bg051.zip : zip already processed
../images/fg_bg/bg052.zip : zip already processed
../images/fg_bg/bg053.zip : zip alread